In [171]:
from osgeo import gdal
import numpy as np
import os
import scipy.ndimage
import scipy.stats
import matplotlib.pyplot as plt
import time
import skimage.measure


In [2]:
def make_raster(in_ds, fn, data, data_type, nodata=None): 
    """Create a one-band GeoTIFF.
    in_ds     - datasource to copy projection and geotransform from
    fn        - path to the file to create
    data      - NumPy array containing data to write
    data_type - output data type
    nodata    - optional NoData value
    """
    driver = gdal.GetDriverByName('GTiff')
    out_ds = driver.Create(fn, in_ds.RasterXSize, in_ds.RasterYSize, 1, data_type)
    out_ds.SetProjection(in_ds.GetProjection()) 
    out_ds.SetGeoTransform(in_ds.GetGeoTransform())
    out_band = out_ds.GetRasterBand(1)
    if nodata is not None:
        out_band.SetNoDataValue(nodata)
        out_band.WriteArray(data)
        out_band.FlushCache()
        out_band.ComputeStatistics(False)
    return out_ds

In [23]:
# Write a function that calculates slope using a 3x3 window.
# This will be passed to the SciPy filter function below.
def slope(data, cell_width, cell_height):
    """Calculates slope using a 3x3 window.

    data        - 1D array containing the 9 pixel values, starting
                  in the upper left and going left to right and down
    cell_width  - pixel width in the same units as the data
    cell_height - pixel height in the same units as the data
    """
    rise = ((data[6] + (2 * data[7]) + data[8]) - (data[0] + (2 * data[1]) + data[2])) / (8 * cell_height)
    run =  ((data[2] + (2 * data[5]) + data[8]) - (data[0] + (2 * data[3]) + data[6])) / (8 * cell_width)
    dist = np.sqrt(np.square(rise) + np.square(run))
    return np.arctan(dist) * 180 / np.pi

In [125]:
# Write a function that calculates 3D area of DEM using a 3x3 window.
# This will be passed to the SciPy filter function below.
def area3D(data, cell_width, cell_height):
    """Calculates area per cell using a 3x3 window and a triangulated regular network.

    data        - 1D array containing the 9 pixel values, starting
                  in the upper left and going left to right and down
    cell_width  - pixel width in the same units as the data
    cell_height - pixel height in the same units as the data
    """
    
    xyz1 = np.array([0*cell_height,1*cell_width,data[1]])
    xyz2 = np.array([0*cell_height,2*cell_width,data[2]])
    xyz3 = np.array([1*cell_height,0*cell_width,data[3]])
    xyz4 = np.array([1*cell_height,1*cell_width,data[4]])
    xyz5 = np.array([1*cell_height,2*cell_width,data[5]])
    xyz6 = np.array([2*cell_height,0*cell_width,data[6]])
    xyz7 = np.array([2*cell_height,1*cell_width,data[7]])
    
    base12 = np.linalg.norm(xyz1 - xyz2)
    base34 = np.linalg.norm(xyz3 - xyz4)
    base45 = np.linalg.norm(xyz4 - xyz5)
    base67 = np.linalg.norm(xyz6 - xyz7)
    
    height14 = np.linalg.norm(xyz1 - xyz4)
    height25 = np.linalg.norm(xyz2 - xyz5)
    height36 = np.linalg.norm(xyz3 - xyz6)
    height47 = np.linalg.norm(xyz4 - xyz7)
    
    t124 = base12*height14/2
    t134 = base34*height14/2
    t245 = base45*height25/2
    t346 = base34*height36/2
    t457 = base45*height47/2
    t467 = base67*height47/2
    
    area = 0.25*(t124+t245+t346+t467)+0.5*(t134+t457)
    
    return area

In [ ]:
# fit plane to DEM in a local window
def fit_plane_local(data, cell_width, cell_height):

    # figure out size of window
    npoints = data.size
    #dim = data.shape[0]
    dim = int(np.sqrt(npoints))
   # print("npoints ", npoints)
   # print("npoints again", npoints)
    # fitting [x y 1][a b c]'=z for multiple rows of x,y,z (at least three)
    # M*A=Z
    
    
    
    M = np.ones((npoints,3)) 
    k = 0
    for i in range(dim):
        for j in range(dim):
            M[k,:] = [j*cell_width, i*cell_height, 1]
            k+=1
  
    #indmatx = np.outer(ones((1,dim)),[range(dim)])
    #indmaty = indmatx.transpose()
    #xvec = indmatx.reshape(npoints,1) * cell_width
    #yvec = indmaty.reshape(npoints,1) * cell_height
    #M = np.concatenate((xvec, yvec, np.ones((npoints,1))), axis=1)
    
    a,b,c = np.linalg.lstsq(M,data)[0]
   # print ("a {}, b {}, c{} ".format(a,b,c))
    #slope_x = a
    #slope_y = b
    #z_intestect = c
    #aspect = np.arctan2(b,a)
    
    #slope = np.linalg.norm([a,b])
   
   # slope = dim
   # print("slope ",slope)
    return complex(a,b)

In [126]:
#in_fn = '/Users/opizarro/data/FK20180119/bathy/hawaii_bty_5m_Au_au_0-400m.tif' # low res version - not 5m
in_fn = '/Users/opizarro/data/FK20180119/bathy/au_au_clipped_UTM_4N.tif'

In [122]:
# all input data
in_ds = gdal.Open(in_fn)
in_data = in_ds.GetRasterBand(1).ReadAsArray()


#cell_width = 5
cell_width = in_ds.GetGeoTransform()[1]
#cell_height = 5 
cell_height = in_ds.GetGeoTransform()[5]
print("cell height ",cell_height)
print("input data shape ",in_data.shape)

('cell height ', -4.9997952662983325)
('input data shape ', (4788, 6705))


In [127]:
# test section of input data
in_ds = gdal.Open(in_fn)

off_ulx = 3300
off_uly = 2000
columns = 1000
rows = 800
in_data = in_ds.GetRasterBand(1).ReadAsArray(off_ulx, off_uly, columns, rows)


#cell_width = 5
cell_width = in_ds.GetGeoTransform()[1]
#cell_height = 5 
cell_height = in_ds.GetGeoTransform()[5]
print("cell height ",cell_height)
print("input data shape ",in_data.shape)

('cell height ', -4.9997952662983325)
('input data shape ', (800, 1000))


In [23]:
# fit plane to DEM in a local window
 # figure out size of window
npoints = 9
    #dim = data.shape[0]
dim = int(np.sqrt(npoints))
   # print("npoints ", npoints)
   # print("npoints again", npoints)
    # fitting [x y 1][a b c]'=z for multiple rows of x,y,z (at least three)
    # M*A=Z
    
    
    
M = np.ones((npoints,3)) 
k = 0
for i in range(dim):
    for j in range(dim):
        M[k,:] = [j*cell_width, i*cell_height, 1]
        k+=1

Pinv = np.linalg.pinv(M)        
print(M)
print(Pinv)
        
def fit_plane_local_M(data, cell_width, cell_height,Pinv):

   
  
    #indmatx = np.outer(ones((1,dim)),[range(dim)])
    #indmaty = indmatx.transpose()
    #xvec = indmatx.reshape(npoints,1) * cell_width
    #yvec = indmaty.reshape(npoints,1) * cell_height
    #M = np.concatenate((xvec, yvec, np.ones((npoints,1))), axis=1)
    
    ##a,b,c = np.linalg.lstsq(M,data)[0]
   # print ("a {}, b {}, c{} ".format(a,b,c))
    #slope_x = a
    #slope_y = b
    #z_intestect = c
    #aspect = np.arctan2(b,a)
    
    #slope = np.linalg.norm([a,b])
   
    theta = np.dot(Pinv,data)
    a = theta[0]
   # slope = dim
   # print("slope ",slope)
    return a

[[ 0.         -0.          1.        ]
 [ 4.99962942 -0.          1.        ]
 [ 9.99925885 -0.          1.        ]
 [ 0.         -4.99979527  1.        ]
 [ 4.99962942 -4.99979527  1.        ]
 [ 9.99925885 -4.99979527  1.        ]
 [ 0.         -9.99959053  1.        ]
 [ 4.99962942 -9.99959053  1.        ]
 [ 9.99925885 -9.99959053  1.        ]]
[[ -3.33358040e-02   6.32959636e-18   3.33358040e-02  -3.33358040e-02
    1.86967513e-18   3.33358040e-02  -3.33358040e-02  -1.19404796e-17
    3.33358040e-02]
 [  3.33346983e-02   3.33346983e-02   3.33346983e-02  -2.23694065e-17
   -7.82245417e-18   1.08849024e-17  -3.33346983e-02  -3.33346983e-02
   -3.33346983e-02]
 [  4.44444444e-01   2.77777778e-01   1.11111111e-01   2.77777778e-01
    1.11111111e-01  -5.55555556e-02   1.11111111e-01  -5.55555556e-02
   -2.22222222e-01]]


In [128]:
# wrapper functon to allow for different window sizes


        
def fit_plane_local_a(data, cell_width, cell_height,Pinv):
    
    theta = np.dot(Pinv,data)
    a = theta[0]
    return a

def fit_plane_local_b(data, cell_width, cell_height,Pinv):
    
    theta = np.dot(Pinv,data)
    b = theta[1]
    return b

def fit_plane_window(in_data, win_size, cell_width, cell_height ):
    
    dim = win_size
    npoints = dim**2
    M = np.ones((npoints,3)) 
    k = 0
    for i in range(dim):
        for j in range(dim):
            M[k,:] = [j*cell_width, i*cell_height, 1]
            k+=1

    Pinv = np.linalg.pinv(M)        

    a = scipy.ndimage.filters.generic_filter(in_data, fit_plane_local_a, size=win_size, mode='nearest', extra_arguments=(
        cell_width, cell_height, Pinv))
    
    b = scipy.ndimage.filters.generic_filter(in_data, fit_plane_local_b, size=win_size, mode='nearest', extra_arguments=(
        cell_width, cell_height, Pinv))
    
    # plane equation is ax + by +c = z
    # or a*(j*cw) + b*(i*ch) + c = z
    slope = np.arctan(np.sqrt(np.square(a)+np.square(b)))*180/np.pi
    aspect = np.arctan2(b,a)*180/np.pi
    # area per horizontal unit area
    areaphua = np.sqrt(np.square(a)+np.square(b)+np.ones(a.shape))
    print(slope.shape)
    print(aspect.shape)
        
    return (slope, aspect, areaphua, a, b)


In [6]:

# test at one scale
win_size = 3
tic = time.tic = time.time()time()
slope3, aspect3 = fit_plane_window(in_data, win_size, cell_width, cell_height)
toc = time.time()
print("shape of slope ", slope.shape)
print("ellapsed time ", toc-tic)
print("cell height ",cell_height)
plt.figure(5)
plt.imshow(slope)
plt.draw()
plt.figure(6)
plt.imshow(aspect)
plt.draw()
plt.show(block=False)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:37: RuntimeWarning: overflow encountered in square


(4788, 6705)
(4788, 6705)
('shape of slope ', (4788, 6705))
('ellapsed time ', 555.2659420967102)
('cell height ', -4.9997952662983325)


In [129]:
# calculate 3D area per cell
tic = time.time()
area3 = scipy.ndimage.filters.generic_filter(in_data, area3D, size=3, mode='nearest', extra_arguments=(
        cell_width, cell_height))
toc = time.time()
print("shape of area3 ",area3.shape)
print("ellapsed time ",toc-tic)
plt.figure(1)
plt.imshow(area3)
plt.draw()
plt.show(block=False)


('shape of area3 ', (800, 1000))
('ellapsed time ', 34.56671714782715)


In [130]:
scales = []
feat = []
for win_size in [3,5,7,9]:
    print("window size ", win_size)
    tic = time.time()
    slope, aspect, areaphua, a, b = fit_plane_window(in_data, win_size, cell_width, cell_height)
   
    area3Dwin = scipy.ndimage.filters.uniform_filter(area3,win_size)
    rugosity = np.divide(area3Dwin,areaphua*cell_width*cell_height)
    
    toc = time.time()
    
    scales.append((win_size,slope, aspect, rugosity, area3Dwin, areaphua, a, b))
    
    feat.append(slope)
    feat.append(aspect)
    feat.append(rugosity)
    #feat.append(np.stack((slope, aspect, rugosity), axis=-1))
    #print("feat shape " ,np.stack((slope, aspect, rugosity), axis=-1).shape)
    print("shape of slope ", slope.shape)
    print("ellapsed time ", toc-tic)
    plt.figure(win_size*5)
    plt.imshow(slope)
    plt.title('slope %s'%(win_size))
    
#    plt.figure(win_size*5+1)
#    plt.imshow(aspect, cmap="hsv")
#    plt.title('aspect %s'%(win_size))
    
#    plt.figure(win_size*5+2)
#    plt.imshow(rugosity)
#    plt.title('rugosity %s'%(win_size))
    
#    plt.figure(win_size*5+3)
#    plt.imshow(area3Dwin)
#    plt.title('area3Dwin %s'%(win_size))
    
#    plt.figure(win_size*5+4)
#    plt.imshow(areaphua)
#    plt.title('areaphua %s'%(win_size))
    
    
plt.show(block=False)

# add depth data
feat.append(in_data)
# stack list in 3 dimensional array
featstack = np.stack(feat,axis=-1)

('window size ', 3)
(800, 1000)
(800, 1000)
('shape of slope ', (800, 1000))
('ellapsed time ', 1.6733429431915283)
('window size ', 5)
(800, 1000)
(800, 1000)
('shape of slope ', (800, 1000))
('ellapsed time ', 1.7282450199127197)
('window size ', 7)
(800, 1000)
(800, 1000)
('shape of slope ', (800, 1000))
('ellapsed time ', 1.8504998683929443)
('window size ', 9)
(800, 1000)
(800, 1000)
('shape of slope ', (800, 1000))
('ellapsed time ', 1.8792529106140137)


In [115]:
#print("in_data shape ",in_data.shape)
#feat= []
#feat.append(in_data)


print("feature stack shape ",featstack.shape)

plt.figure(5)
nplots = featstack.shape[2]
f, faxarr = plt.subplots(4,1)
g, gaxarr = plt.subplots(4,1)
h, haxarr = plt.subplots(4,1)


for k in range(4):
    faxarr[k].hist(featstack[:,:,3*k].flatten(),bins=50)
    gaxarr[k].hist(featstack[:,:,3*k+1].flatten(),bins=50)
    haxarr[k].hist(featstack[:,:,3*k+2].flatten(),bins=np.arange(0.9,1.5,0.02))

plt.show(block=False)




('feature stack shape ', (800, 1000, 13))


In [137]:

import spectral


proc_data=in_data[:,:,np.newaxis]

print( "proc_data shape", proc_data.shape )
classes, centres = spectral.kmeans(proc_data, 15, 40)
print("classes shape ", classes.shape)
plt.figure(10)
plt.imshow(classes, cmap = "nipy_spectral")
plt.figure(11)
plt.imshow(in_data, cmap = "nipy_spectral" )
plt.show(block=False)

('proc_data shape', (800, 1000, 1))
Initializing clusters along diagonal of N-dimensional bounding box.
Iteration 1...  0.0%Iteration 1...799999 pixels reassigned.
Iteration 2...  0.0%Iteration 2...55420 pixels reassigned.
Iteration 3...  0.0%Iteration 3...43625 pixels reassigned.
Iteration 4...  0.0%Iteration 4...36471 pixels reassigned.
Iteration 5...  0.0%Iteration 5...29847 pixels reassigned.
Iteration 6...  0.0%Iteration 6...23750 pixels reassigned.
Iteration 7...  0.0%Iteration 7...21999 pixels reassigned.
Iteration 8...  0.0%Iteration 8...19411 pixels reassigned.
Iteration 9...  0.0%Iteration 9...16075 pixels reassigned.
Iteration 10...  0.0%Iteration 10...13725 pixels reassigned.
Iteration 11...  0.0%Iteration 11...11603 pixels reassigned.
Iteration 12...  0.0

In [165]:
# downsample to make problem more tractable


def mode_win(data,axis):
    return scipy.stats.mstats.mode(data,axis)[0]

from skimage.measure import block_reduce
tic = time.time()
#mode_data = scipy.ndimage.filters.generic_filter(classes, mode_win, size = 11)
mode_data = block_reduce(classes, block_size=(10,10), func = mode_win )
toc = time.time()
print("mode block time ",toc-tic)
print("mode_data shape", mode_data.shape)

('mode block time ', 70.68977093696594)
('mode_data shape', (80, 100, 10, 1))


In [170]:

plt.figure(13)
plt.imshow(mode_data[:,:,0,0], cmap = "nipy_spectral")
plt.show(block=False)

In [210]:
# region labeling with scikit
label_image, rnum = skimage.measure.label(mode_data[:,:,0,0],connectivity=2,return_num=True)
print("number of regions ", rnum)
plt.figure(113)
plt.imshow(label_image, cmap = "prism")
plt.show(block=False)

('number of regions ', 565)


In [211]:
tic = time.time()
properties = skimage.measure.regionprops(label_image)
toc = time.time()
print("properties calc time {} for {} regions ".format(toc-tic,rnum))
print("size of properties {}".format(len(properties)))
print(properties[500].centroid)

properties calc time 0.0110669136047 for 565 regions 
size of properties 565
(60.0, 54.0)


In [213]:
plt.figure(114)
plt.imshow(label_image, cmap = "prism")

for ri in range(len(properties)):
    plt.scatter(x=properties[ri].centroid[1],y=properties[ri].centroid[0],c='k',s=10)
plt.show(block=False)
    

In [ ]:
# assemble generalised TSP


# solve GTSP



In [17]:
# fit plane to DEM in a local window
# a faster option would precompute matrix M as a function of the window size
# and pass it as an argument
# to return two values, could try using complex number

def fit_plane_local_multioutput(data, cell_width, cell_height,outlist):

    # figure out size of window
    npoints = data.size
    dim = int(np.sqrt(npoints))
    
    
    
    M = np.ones((npoints,3))
   
    k = 0
    for i in range(dim):
        for j in range (dim):
            M[k,:] = [j*cell_width, i*cell_height, 1]
    #        Z[k] = data[k]
            k+=1
  
    a,b,c = np.linalg.lstsq(M,data)[0]
   
    aspect = np.arctan2(b,a)*180/np.pi
    #outvec.extend(aspect)
    outlist.append(aspect)
    slope = np.arctan(np.linalg.norm([a,b]))
   
   # slope = dim
   # print("slope ",slope)
    return slope

In [54]:
plt.figure(1)
plt.imshow(in_data)

out_data = scipy.ndimage.filters.uniform_filter( in_data, size=3, mode='nearest')
plt.figure(2)
plt.imshow(out_data)
plt.draw()
#plt.show()
#pb.make_raster(in_ds, out_fn, out_data, gdal.GDT_Int32) del in_ds

In [11]:
slope3 = scipy.ndimage.filters.generic_filter(in_data, slope, size=3, mode='nearest', extra_arguments=(
        cell_width, cell_height))
print("size of slope3 ",slope3.size)
print("cell height ",cell_height)
plt.figure(3)
plt.imshow(slope3)
plt.draw()

('size of slope3 ', 300000)
('cell height ', -4.9997952662983325)


In [39]:
area3 = scipy.ndimage.filters.generic_filter(in_data, area3D, size=3, mode='nearest', extra_arguments=(
        cell_width, cell_height))
print("size of area3 ",area3.size)
print("cell height ",cell_height)
plt.figure(4)
plt.imshow(area3)
plt.draw()

('size of area3 ', 300000)
('cell height ', -4.9997952662983325)


In [19]:
tic = time.time()

slope_plane = scipy.ndimage.filters.generic_filter(in_data, fit_plane_local_M, size=3, mode='nearest', extra_arguments=(
        cell_width, cell_height, Pinv))
toc = time.time()
print("size of area3 ", slope_plane.size)
print("ellapsed time ", toc-tic)
print("cell height ",cell_height)
plt.figure(5)
plt.imshow(abs(slope_plane)*180/np.pi)
plt.draw()
plt.show()

('size of area3 ', 300000)
('ellapsed time ', 3.2434260845184326)
('cell height ', -4.9997952662983325)


In [31]:
plt.figure(7)
plt.imshow(slope_plane.imag)
plt.draw()
plt.figure(8)
plt.imshow(slope_plane.real)
plt.draw()
plt.show()

In [20]:
outlist = []
tic = time.time()
slope_plane = scipy.ndimage.filters.generic_filter(in_data, fit_plane_local_multioutput, size=3, mode='nearest', extra_arguments=(
        cell_width, cell_height,outlist))
toc = time.time()
print("length of outvec", len(outlist))
print("shape of area3 ", slope_plane.shape)
print("ellapsed time ", toc-tic)
print("cell height ",cell_height)
plt.figure(5)
plt.imshow(slope_plane)
plt.draw()



('length of outvec', 300000)
('shape of area3 ', (500, 600))
('ellapsed time ', 357.0875198841095)
('cell height ', -4.9997952662983325)


In [24]:
outmat = np.array(outlist).reshape(slope_plane.shape);
print(outmat.shape)

plt.figure(5)
plt.imshow(slope_plane)
plt.draw()


plt.figure(6)
plt.imshow(outmat)
plt.draw()
plt.show()



(500, 600)


KeyboardInterrupt: 